In [2]:
import ipywidgets as widgets
from prettytable import PrettyTable
from fastai.learner import load_learner
import pickle
import urllib.request

MODEL_URL = "https://drive.google.com/uc?export=download&id=1dCinFMc2O7oPLQnMS-GGcQWv-gyKYHQQ"
urllib.request.urlretrieve(MODEL_URL, "model.pkl")

learn = load_learner("model.pkl")
labels = learn.dls.vocab[1]

def jupyter_serve(max_n=5):
    string = text_input.value
    output_grid = PrettyTable(format=True)
    output_grid.field_names = ["Emoji", "Confidence"]
    confidences = learn.predict(string)[2].tolist() # Get List of predictions from fastai
    results = {l: c for l, c in zip(labels, confidences)} # Wrap labels and confidences into dictionary
    results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    top_results = results[:5]
    total_confidence = sum([x[1] for x in top_results])
    for row in top_results:
        output_grid.add_row(row)
    lbl_pred.value = output_grid.get_html_string()

text_input = widgets.Textarea(
    value='',
    placeholder='I love baseball, but the games can be soo boring',
    disabled=False
)

lbl_pred = widgets.HTML()
lbl_pred.value = ""

btn_run = widgets.Button(description='Classify')
btn_run.on_click(jupyter_serve)


widgets.VBox([widgets.Label('Enter your message!'), 
      text_input, btn_run, lbl_pred])

ModuleNotFoundError: No module named 'prettytable'